In [1]:
using Rocket
using BenchmarkTools

In [2]:
n = 100000

array      = collect(1:n)
observable = from(array);

In [3]:
reactive(source::S) where S = check_reactive(as_subscribable(S), source)

check_reactive(::SimpleSubscribableTrait{T}, source)    where T = _reactive(T, source)
check_reactive(::ScheduledSubscribableTrait{T}, source) where T = _reactive(T, source)

function _reactive(::Type{T}, source) where T
    keep = KeepActor{T}()
    subscribe!(source, keep)
    return keep.values
end

_reactive (generic function with 1 method)

# Map operator performance comparison

In [4]:
function regularArrayMap(source)
    keep = Vector{Int}()
    for v in source
        push!(keep, v ^ 2)
    end
    return keep
end

observable_mapped = observable |> map(Int, (d) -> d ^ 2)

println(regularArrayMap(array) == reactive(observable_mapped))

@btime regularArrayMap($array);
@btime reactive($observable_mapped);

true
  990.879 μs (17 allocations: 2.00 MiB)
  958.021 μs (17 allocations: 2.00 MiB)


# Filter operator performance comparison

In [5]:
function regularArrayFilter(source)
    keep   = Vector{Int}()
    for v in source
        if v % 2 == 0
            push!(keep, v)
        end
    end
    return keep
end

observable_filtered = observable |> filter((d) -> d % 2 == 0)

println(regularArrayFilter(array) == reactive(observable_filtered))

@btime regularArrayFilter($array);
@btime reactive($observable_filtered);

true
  414.523 μs (16 allocations: 1.00 MiB)
  391.378 μs (16 allocations: 1.00 MiB)


# Enumerate operator performance comparison

In [6]:
function regularArrayEnumerate(source)
    keep   = Vector{Tuple{Int, Int}}()
    current = 1
    for v in source
        push!(keep, (v, current))
        current += 1
    end
    return keep
end

observable_enumerated = observable |> enumerate()

println(regularArrayEnumerate(array) == reactive(observable_enumerated))

@btime regularArrayEnumerate($array);
@btime reactive($observable_enumerated);

true
  1.342 ms (17 allocations: 3.00 MiB)
  1.430 ms (18 allocations: 3.00 MiB)


# Scan operator performance comparison

In [7]:
function regularArrayScan(source, seed::Int)
    keep    = Vector{Int}()
    current = seed
    for v in source
        current = current + v
        push!(keep, current)
    end
    return keep
end

observable_scanned = observable |> scan(Int, +, 0)

println(regularArrayScan(array, 0) == reactive(observable_scanned))

@btime regularArrayScan($array, $0);
@btime reactive($observable_scanned);

true
  971.512 μs (17 allocations: 2.00 MiB)
  989.223 μs (18 allocations: 2.00 MiB)


# Reduce operator performance comparison

In [8]:
function regularArrayReduce(source, seed::Int)
    keep    = Vector{Int}()
    current = seed
    for v in source
        current = current + v
    end
    push!(keep, current)
    return keep
end

observable_reduced = observable |> reduce(Int, +, 0)

println(regularArrayReduce(array, 0) == reactive(observable_reduced))

@btime regularArrayReduce($array, $0);
@btime reactive($observable_reduced);

true
  11.402 μs (2 allocations: 128 bytes)
  33.371 μs (3 allocations: 160 bytes)


# Max operator performance comparison

In [9]:
function regularArrayMax(source)
    keep    = Vector{Int}()
    current = nothing::Union{Int, Nothing}
    for v in source
        if current == nothing
            current = v
        else
            if v > current
                current = v
            end
        end
    end
    push!(keep, current)
    return keep
end

observable_max = observable |> max()

println(regularArrayMax(array) == reactive(observable_max))

@btime regularArrayMax($array);
@btime reactive($observable_max);

true
  140.014 μs (2 allocations: 128 bytes)
  54.831 μs (3 allocations: 160 bytes)


# Min operator performance comparison

In [10]:
function regularArrayMin(source)
    keep    = Vector{Int}()
    current = nothing::Union{Int, Nothing}
    for v in source
        if current == nothing
            current = v
        else
            if v < current
                current = v
            end
        end
    end
    push!(keep, current)
    return keep
end

observable_min = observable |> min()

println(regularArrayMin(array) == reactive(observable_min))

@btime regularArrayMin($array);
@btime reactive($observable_min);

true
  140.023 μs (2 allocations: 128 bytes)
  47.924 μs (3 allocations: 160 bytes)


# Sum operator performance comparison

In [11]:
function regularArraySum(source)
    keep    = Vector{Int}()
    current = nothing::Union{Int, Nothing}
    for v in source
        if current == nothing
            current = v
        else
            current = current + v
        end
    end
    push!(keep, current)
    return keep
end

observable_sum = observable |> sum()

println(regularArraySum(array) == reactive(observable_sum))

@btime regularArraySum($array);
@btime reactive($observable_sum);

true
  93.782 μs (2 allocations: 128 bytes)
  126.977 μs (3 allocations: 160 bytes)
